In [20]:
import minsearch
import os
import numpy as np
from groq import Groq
from dotenv import load_dotenv
import json
from qdrant_client import QdrantClient,models
from fastembed import TextEmbedding

In [2]:
client= QdrantClient("http://localhost:6333") # Connecting to local Qdrant Instance

In [3]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [4]:
model = TextEmbedding(model_name=model_handle)

In [5]:
string = 'I just discovered the course. Can I join now?'

In [6]:
embedding = np.array((list(model.embed(string))))

In [7]:
min_val = np.min(embedding)

In [8]:
min_val

np.float64(-0.11726373885183883)

In [9]:
doc = 'Can I still join the course after the start date?'

In [10]:
embedding_2 = np.array((list(model.embed(doc))))

In [11]:
query = "I just discovered the course. Can I join now?"
doc = "Can I still join the course after the start date?"

# Get embeddings
query_vec = next(model.embed([query]))
doc_vec = next(model.embed([doc]))

In [12]:
cosine_similarity = np.dot(query_vec, doc_vec)
print(cosine_similarity)

0.9008528895674548


In [13]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [14]:
similarities = []

for doc in documents:
    text_embed = next(model.embed(doc["text"]))
    query_embed = next(model.embed(doc["question"]))
    similarity = text_embed.dot(query_embed)
    
    similarities.append({
        "question": doc["question"],
        "answer": text_embed,
        "similarity": similarity
    })


most_similar_index = max(
    range(len(similarities)),
    key=lambda i: similarities[i]["similarity"]
)

print("Most similar document index:", most_similar_index)


Most similar document index: 1


In [15]:
similarities_2 = []

for doc in documents:
    full_text = doc["question"] + " " + doc["text"]
    full_text_embed =  next(model.embed(full_text))
    query_embed = next(model.embed(doc["question"]))
    similarity = full_text_embed.dot(query_embed)
    
    similarities_2.append({
        "question": doc["question"],
        "answer": text_embed,
        "similarity": similarity
    })


most_similar_index = max(
    range(len(similarities_2)),
    key=lambda i: similarities_2[i]["similarity"]
)

print("Most similar document index:", most_similar_index)


Most similar document index: 1


In [16]:
_models = TextEmbedding.list_supported_models()

smallest_model = min(_models, key=lambda m: m["dim"])

print(json.dumps(smallest_model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}


In [17]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [27]:
collection_name = "homework-rag2"
EMBEDDING_DIMENSIONALITY=384
model_handle='BAAI/bge-small-en'

In [28]:

client.create_collection(
collection_name=collection_name,
vectors_config=models.VectorParams(
    size=EMBEDDING_DIMENSIONALITY, #Dimensionality of the vectors
    distance=models.Distance.COSINE # Distance metric for similarity search
)
)

True

In [29]:
points = []
id = 0

for course in documents:
    
    point = models.PointStruct(
        id=id,
    vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_handle), 
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [30]:
client.upsert(
    collection_name=collection_name,
    points=points
)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\admin_ABH\anaconda3\envs\ai-engineering\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin_ABH\AppData\Local\Temp\2\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 files: 100%|████

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [32]:
search(string)

QueryResponse(points=[ScoredPoint(id=66, version=0, score=0.80676496, payload={'text': 'I’ve seen LinkedIn users list DataTalksClub as Experience with titles as:\nMachine Learning Fellow\nMachine Learning Student\nMachine Learning Participant\nMachine Learning Trainee\nPlease note it is best advised that you do not list the experience as an official “job” or “internship” experience since DataTalksClub did not hire you, nor financially compensate you.\nOther ways you can incorporate the experience in the following sections:\nOrganizations\nProjects\nSkills\nFeatured\nOriginal posts\nCertifications\nCourses\nBy Annaliese Bronz\nInteresting question, I put the link of my project into my CV as showcase and make posts to show my progress.\nBy Ani Mkrtumyan', 'section': 'Miscellaneous', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])